In [1]:
import glob
import pandas as pd
import json

maps_dir = ['map 0', 'map 1' , 'map 2', 'map 3', 'map 4']
map_dir = 'map 4'
path = './data/'+map_dir+'/data-of-baselines.json'

In [2]:
#load data from the json file
with open(path) as train_file:
    data = json.load(train_file)

participants_df = pd.DataFrame.from_dict(data['all-games'], orient='index')

In [3]:
li = []
for map_dir in maps_dir:
    path = 'data/'+map_dir+'/workers' # use your path
    all_files = glob.glob(path + "/*.csv")
    for filename in all_files:
        fromOne = pd.read_csv(filename, index_col=None, header=0)
        fromOne['filename'] = filename
        fromOne['map'] = map_dir
        li.append(fromOne)

workers = pd.concat(li, axis=0, ignore_index=True)
workers = workers.reset_index()
print(workers.index[workers.index.duplicated()])
workers = workers[~workers.index.duplicated(keep='first')]

df = workers[['WorkerId', 'map']]
df

Int64Index([], dtype='int64')


,WorkerId,map
0,A12XVSIL669PVI,map 0
1,A1WJIRT360P5VB,map 0
2,A3RYI5HXC2MJLN,map 0
3,AUR5KDE75HSZ9,map 0
4,AWH9PNZE79UCX,map 0
...,...,...
1230,A2C4R0TBPZGWTD,map 4
1231,A2WYUG5PTNXZNT,map 4
1232,A3LTGXX5HIYDBM,map 4
1233,A3V0ZNHKABDUMM,map 4


In [4]:
df_sub = df[df['map'] == map_dir]
df = df.drop(df_sub.index)
# df_sub = df_sub.drop('map', axis=1)
df_sub = df_sub.drop_duplicates(keep='first')

In [5]:
df1 = df.groupby('WorkerId').count()
df2 = df_sub.groupby('WorkerId').count()
df_new = df1.join(df2, on='WorkerId', how='inner', lsuffix='_left', rsuffix='_right')

In [6]:
df_new = df_new[df_new['map_left'].notna()]
df_new = df_new.reset_index()
# df_new['map'].sum()
df_new

,WorkerId,map_left,map_right
0,A1AP3M61EEPHC7,2,1
1,A1IKTS2RGE9D0T,1,1
2,A1IKWVZZ26QTLU,2,1
3,A1M9ED6NUDC9KV,1,1
4,A1RUBSQBBOJHPO,3,1
5,A1TWVJS27CL3KT,3,1
6,A2196PMIF2YORD,3,1
7,A23DIRTLAL5RA6,13,1
8,A2AJ5QFNC9KF43,1,1
9,A2EIQYUSCVZTML,1,1


In [7]:
participants_df = pd.DataFrame.from_dict(data['all-games'], orient='index')

path = 'data/'+map_dir+'/workers' # use your path
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    fromOne = pd.read_csv(filename, index_col=None, header=0)
    fromOne['filename'] = filename
    li.append(fromOne)

workers = pd.concat(li, axis=0, ignore_index=True)
workers = workers.set_index('Answer.surveycode')
workers.index = workers.index.map(lambda code: '-' + code[:-3])
print(workers.index[workers.index.duplicated()])
workers = workers[~workers.index.duplicated(keep='first')]
participants_df['WorkerId'] = workers['WorkerId']
participants_df = participants_df.dropna(subset = ['WorkerId'])
participants_df = participants_df[(~participants_df.duplicated('WorkerId')) | (participants_df['WorkerId'].isnull())]

Index([], dtype='object', name='Answer.surveycode')


In [8]:
indexes_drop = []
for id in df_new['WorkerId']:
    index = participants_df[participants_df.WorkerId == id].index
    indexes_drop.append(index.to_numpy()[0])

participants_df = participants_df.drop(indexes_drop)
participants_df.loc[: ,'human_score'] = pd.to_numeric(participants_df['human_score'], errors='coerce')
participants_df.loc[: ,'computer_score'] = pd.to_numeric(participants_df['computer_score'], errors='coerce')
participants_df.loc[: ,'collaborative_value'] = pd.to_numeric(participants_df['collaborative_value'], errors='coerce')
participants_df.loc[: ,'predictable_value'] = pd.to_numeric(participants_df['predictable_value'], errors='coerce')
participants_df.loc[: ,'selfishly_value'] = pd.to_numeric(participants_df['selfishly_value'], errors='coerce')
participants_df.loc[: ,'wisely_value'] = pd.to_numeric(participants_df['wisely_value'], errors='coerce')

In [11]:
participants_df['behavior'].value_counts()

closest     16
farthest    12
TSP         10
selfish      9
random       6
Name: behavior, dtype: int64

In [10]:
participants_df.groupby('behavior')[['computer_score', 'human_score', 'collaborative_value', 'predictable_value', 'selfishly_value', 'wisely_value']].mean()

,computer_score,human_score,collaborative_value,predictable_value,selfishly_value,wisely_value
behavior,,,,,,
TSP,0.930000,0.859000,5.100000,4.800000,3.200000,5.600000
closest,0.915625,0.927500,5.062500,5.187500,2.875000,5.437500
farthest,0.507500,0.615000,5.000000,5.166667,5.166667,4.916667
random,0.553333,0.521667,2.333333,4.500000,5.500000,3.500000
selfish,1.217778,0.426667,3.666667,5.888889,6.222222,4.111111
